In [ ]:
! pip install pydub edge_tts webrtcvad git+https://github.com/openai/whisper.git

In [2]:
# Import necessary libraries
import whisper
import torch
from transformers import pipeline
import edge_tts
from pydub import AudioSegment
import webrtcvad
import numpy as np
import wave

In [3]:
# Step 1: Voice-to-Text Conversion
def voice_to_text(audio_file_path):
    # Load Whisper model
    model = whisper.load_model("base.en")

    # Perform speech-to-text
    result = model.transcribe(audio_file_path, language='en')
    text = result['text']
    return text

In [4]:
# Frame generator function for VAD
def frame_generator(frame_duration_ms, audio, sample_rate):
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    while offset + n < len(audio):
        yield audio[offset:offset + n]
        offset += n


In [5]:
# Step 2: Text Input into LLM
def query_llm(text):
    # Use Hugging Face transformers pipeline with a pre-trained model
    llm = pipeline("text-generation", model="gpt2")  # Replace with preferred LLM
    response = llm(text, max_length=50, num_return_sequences=1)

    # Restrict the output to a maximum of 2 sentences
    sentences = response[0]['generated_text'].split('. ')
    response_text = '. '.join(sentences[:2])
    return response_text



In [6]:
# Step 3: Text-to-Speech Conversion
import asyncio
async def amain(text, voice='male', pitch=1.0, speed=1.0, output_file='output.mp3'):
    # Convert text to speech with tunable parameters
    communicator = edge_tts.Communicate(text, voice)
    await communicator.save(output_file)
    return output_file

In [11]:
response_text = 0
async def main():
    global response_text
    audio_file = "/content/girl-voice-answering-phone-100740.wav"  # Replace with your audio file path
    text = voice_to_text(audio_file)  # Assuming voice_to_text is defined elsewhere
    print("Recognized Text:", text)

    response_text = query_llm(text)  # Assuming query_llm is defined elsewhere
    print("LLM Response:", response_text)

    output_audio_file = await amain(response_text, voice='en-US-GuyNeural', pitch="+0Hz", speed="+0%")
    print("Speech synthesized to:", output_audio_file)

# Properly run the async main function
try:
    loop = asyncio.get_running_loop()
except RuntimeError:  # No running loop
    loop = None

if loop and loop.is_running():
    # If we're in an environment with a running loop (like Jupyter), use the loop to create a task
    task = loop.create_task(main())
else:
    # If there's no loop running, we can safely use asyncio.run()
    asyncio.run(main())

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


In [12]:
response_text

" Hello? Is there somebody there? Can you hear me? You're scaring me. Who's you? I thought you were just joking"